In [3]:
import joblib
import os
import boto3
import sagemaker

from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

In [ ]:
bucket = sagemaker.Session().default_bucket()
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

In [ ]:
print(role)
print(region)
print(bucket)
print(bucket_path)

In [15]:
model_file_name = "latest-model"

In [16]:
model = joblib.load(model_file_name)

In [14]:
model._Booster.save_model(model_file_name)
!tar czvf model.tar.gz $model_file_name

a latest-model


In [ ]:
#### prefix in S3
prefix = "crop/deployement"

fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")
print(key)
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, "xgboost", "1.3-1")

In [ ]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)

sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])

In [ ]:
from time import gmtime, strftime

endpoint_config_name = "DEMO-XGBoostEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print(endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

In [ ]:
%%time
import time

endpoint_name = "crop-recco-Endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

In [ ]:
runtime_client = boto3.client('runtime.sagemaker')

In [ ]:
%%time
import json


file_name = (
    "test_point.csv"
)
df_x_test.to_csv(file_name,index = False)

with open(file_name, "r") as f:
    next(f)
    payload = f.read()
    

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)



print("Results :\n")
print()

result = response["Body"].read().decode("ascii")

# print("\nPredicted Class Probabilities: {}.".format(result))

In [ ]:
result_lol = '['+result+']'

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)